In [1]:
import requests
import pandas as pd
from time import sleep
from tqdm.notebook import tqdm

In [2]:
ibdata = pd.read_csv('ibdata_Abhir.csv')
ibdata.dropna(subset=["IB-ISIN"],inplace=True)

In [3]:
figi_data=pd.DataFrame(columns=['figi', 'name', 'ticker', 'exchCode', 'compositeFIGI', 'securityType',
       'marketSector', 'shareClassFIGI', 'securityType2',
       'securityDescription'])

empty= {'figi': '',
  'name': '',
  'ticker': '',
  'exchCode': '',
  'compositeFIGI': '',
  'securityType': '',
  'marketSector': '',
  'shareClassFIGI': '',
  'securityType2': '',
  'securityDescription': ''}

In [4]:
openfigi_apikey = '71aaa940-3567-4040-bce5-9f43490a7703'
openfigi_url = 'https://api.openfigi.com/v2/mapping'
openfigi_headers = {'Content-Type': 'text/json'}

In [5]:
def map_jobs(jobs):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=jobs)
    if response.status_code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return response.json()

In [6]:
for i in tqdm(range(920)):
    isins = ibdata['IB-ISIN'][25*i:25*i+25]
    jobs=[]
    for n in isins:
        newjob = {'idType': 'ID_ISIN', 'idValue': n},
        jobs.extend(newjob)
    job_results = map_jobs(jobs)
    just_dictionaries=[]
    for j in job_results:
        try:
            just_dictionaries.append(j['data'][0])
        except:
            just_dictionaries.append(empty)
    figi = pd.DataFrame.from_dict(just_dictionaries)
    figi_data=pd.concat([figi_data,figi])
    sleep(6)

  0%|          | 0/920 [00:00<?, ?it/s]

In [7]:
figi_data

,figi,name,ticker,exchCode,compositeFIGI,securityType,marketSector,shareClassFIGI,securityType2,securityDescription,uniqueID,uniqueIDFutOpt
0,BBG008HNLRF4,1ST GROUP LTD,1ST,AU,BBG008HNLRF4,Common Stock,Equity,BBG008HNLRG3,Common Stock,1ST,NaN,NaN
1,BBG001KS9450,2U INC,TWOU,US,BBG001KS9450,Common Stock,Equity,BBG001V0RR30,Common Stock,TWOU,NaN,NaN
2,BBG000C129N6,3DX INDUSTRIES INC,DDDX,US,BBG000C129N6,Common Stock,Equity,BBG001STRH79,Common Stock,DDDX,NaN,NaN
3,BBG000BP52R2,3M CO,MMM,US,BBG000BP52R2,Common Stock,Equity,BBG001S5T7X2,Common Stock,MMM,NaN,NaN
4,BBG00L33THJ4,MARLEY SPOON AG-CDI,MMM,AU,BBG00L33THJ4,CDI,Equity,BBG00L33THR5,Depositary Receipt,MMM,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9,BBG007BBS8B7,ZYNERBA PHARMACEUTICALS INC,ZYNE,US,BBG007BBS8B7,Common Stock,Equity,BBG007BBS8C6,Common Stock,ZYNE,None,None
10,BBG000BJBXZ2,ZYNEX INC,ZYXI,US,BBG000BJBXZ2,Common Stock,Equity,BBG001S7T7V0,Common Stock,ZYXI,None,None
11,BBG005YZL376,ZYNGA INC - CL A,ZNGAEUR,XS,BBG005YZL2R6,Common Stock,Equity,BBG001T24RN2,Common Stock,ZNGAEUR,None,None
12,BBG000BZPV76,ZYTRONIC PLC,ZYT,LN,BBG000BZPTJ8,Common Stock,Equity,BBG001SFLP13,Common Stock,ZYT,None,None


In [9]:
figi_data=figi_data.add_prefix("OpenFigi-")
figi_data.to_csv("openfigidata_Abhir.csv")